# FACS Analysis

This Jupyter notebook walks you through the basic steps of analyzing Mass-Titr FACS results. For high-throughput binding experiments, the `highThroughputScripts` pipeline is available on [GitHub](https://github.com/KeatingLab/highThroughputScripts/tree/vs_optimization).

To complete the analysis, run all cells in order one-by-one unless otherwise specified, filling in or changing input values as needed. (Hint: Use `Shift+Enter` to run a cell then advance to the next one.) To export a plot, you may want to change the DPI in the `plt.figure` command, i.e. `plt.figure(..., dpi=160)`.

**Requirements**: This notebook requires Python 2, and the following modules:

* `FlowCytometryTools` (install using `pip install flowcytometrytools`)
* `lmfit`
* `ipywidgets`, version 7.2 or later (upgrade if necessary)
* The script `facs_utils.py` must be in the same directory as this notebook.

After installing these modules, be sure to restart the notebook kernel to make sure the modules are available.

*Written by*: Venkatesh Sivaraman, February 2019, adapted from scripts by Dustin Whitney and Theresa Hwang

In [ ]:
### Experiment metadata
# Name:
# Date:
# Info:

In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline
import os
from FlowCytometryTools import FCMeasurement
from FlowCytometryTools import ThresholdGate, PolyGate
import pandas as pd 
from lmfit import Model, Parameters, Minimizer
from facs_utils import *
from collections import OrderedDict
from FlowCytometryTools.core.transforms import hlog

In [ ]:
# Dictionary that contains gating information (rerun this cell to reset interactive gate info)
GATE_RESULTS = {}

# Boundary value for hyperlog transformation
HYPERLOG_B = 100.0

## Step 0. Preliminaries

First we need to provide some information about the experiment, and point the script to where the data is located.

In [ ]:
# Path to directory containing .fcs files
DATA_PATH = "../FACS_data/"

Define and label your titrations here, along with the concentrations at which the experiment was performed. 

Because there are many different methods of plate organization and file naming, you must use the `make_titration` function to tell the script which fcs files to draw data from for each titration. The function takes as parameters the **specimen number**, the **letter code**, and the **number code** for each stop. Each parameter can also be a list, range, or string whose length is the number of concentrations. *Remember that the `range` function's upper bound is non-inclusive, so `range(1, 13)` produces the values [1,2,...,12].*

**Examples:**
* For the command `make_titration(1, 'ABCDEFGH', 1)`, the following wells from Specimen 1 will be used: A1, B1, C1, D1, E1, F1, G1, H1.
* For the command `make_titration(2, 'B', range(1, 7)`, wells B1-B6 prefixed with Specimen 2 will be used.

In [ ]:
CONCENTRATIONS = np.array([20, 5, 1.25, .312, 0.08, 0.01])

TITRATIONS = [
    # The behavior implemented here is to divide each letter code into two sets (1-6 and 7-12),
    # each of which constitutes one titration. Adjust this as needed based on your 
    # experimental setup.
    make_titration(1, 'A', range(1, 7)),
    make_titration(1, 'A', range(7, 13)),
    make_titration(1, 'B', range(1, 7)),
    make_titration(1, 'B', range(7, 13)),
    make_titration(1, 'C', range(1, 7)),
    make_titration(1, 'C', range(7, 13)),
    make_titration(1, 'D', range(1, 7)),
    make_titration(1, 'D', range(7, 13)),
    make_titration(1, 'E', range(1, 7)),
    make_titration(1, 'E', range(7, 13)),
    make_titration(1, 'F', range(1, 7)),
    make_titration(1, 'F', range(7, 13)),
    make_titration(1, 'G', range(1, 7)),
    make_titration(1, 'G', range(7, 13)),
    make_titration(1, 'H', range(1, 7)),
    make_titration(1, 'H', range(7, 13))
]

# Add a descriptive label here for each titration you listed above (e.g. indicate the peptide being tested).
LABELS = [
    "Sample 1", "Sample 2", "Sample 3", "Sample 4",
    "Sample 5", "Sample 6", "Sample 7", "Sample 8",
    "Sample 9", "Sample 10", "Sample 11", "Sample 12",
    "Sample 13", "Sample 14", "Sample 15", "Sample 16"
]

assert all([len(t) == len(CONCENTRATIONS) for t in TITRATIONS]), "All titrations must have same number of concentrations as CONCENTRATIONS list"
assert len(LABELS) == len(TITRATIONS), "Need {} labels to match titrations list, found {}".format(len(TITRATIONS), len(LABELS))

In [ ]:
# Defining axes for various types of gates
scatter_axes_1 = ["FSC-H", "SSC-H"]
scatter_axes_2 = ["SSC-H", "SSC-W"]
fluor_axes = ["Alexa Fluor 680-A", "PE-A"]

# These axes can be hyperlog-transformed if specified
transformable_axes = ["FSC-H", "SSC-H", "SSC-W", "Alexa Fluor 680-A", "PE-A"]

# Expression axis
expression_axis = "PE-A"

In [ ]:
# Utility functions to get sample paths and data

def get_titration_files(titration):
    """Returns the list of fcs files in DATA_PATH corresponding to the given list of concentration stops."""
    return [get_sample_path(*stop) for stop in titration]

def get_sample_path(specimen_number, letter, sample_number):
    """Gets the path for the sample with the given specimen number, letter code, and sample number for the letter code."""
    prefix = 'Specimen_' + str(specimen_number).zfill(3) + '_' + letter + str(sample_number) + '_'
    
    paths = [os.path.join(DATA_PATH, path) for path in os.listdir(DATA_PATH) if path.startswith(prefix)]
    assert len(paths) > 0, "No path found for ({}, {}, {})".format(specimen_number, letter, sample_number)
    assert len(paths) == 1, "Multiple paths satisfy sample path condition"
    return paths[0]

def transform_sample(sample):
    """Performs the standard hyperlog transformation on the given sample."""
    return sample.transform('hlog', channels=transformable_axes, b=HYPERLOG_B)
    
def get_sample(path, id_name, transform=False):
    """Gets a measurement from the given fcs file path, transforming its scatter values using a hyperlog 
    transformation if specified."""
    sample = FCMeasurement(ID=id_name, datafile=path)
    if transform:
        return transform_sample(sample)
    return sample

## Step 1. Forward-Scatter and Side-Scatter Gating

In [ ]:
# Get sample files to test the scatter gates on

test_files = get_titration_files(TITRATIONS[0])
test_samples = [get_sample(path, "Test", transform=True) for path in test_files]

test_file = test_files[0]
test_sample = test_samples[0]

### 1(a). Drawing Gates
**NOTE**: The following two cells utilize the interactive gate drawing tool, the first to draw a gate on the FSC-H/SSC-H plot, and the second to gate the SSC-H/SSC-W plot. If no polygon gate is desired, simply skip over the cell without running it. Alternatively, if you already know the vertices for the desired gates, skip to section 1(b) and replace the value of either `gate_vertices_1` or `gate_vertices_2` with the list of coordinates you want.

In [ ]:
# Choose the number of vertices to use on the gate.
num_points = 3

# Opens the interactive gate drawing tool.
# Use test_samples to plot all 'A' samples together, or test_sample to plot just the first sample (may be faster).
vertex_control(test_samples, scatter_axes_1, num_points, GATE_RESULTS, 'scatter_gate_1', log=False)

In [ ]:
# Choose the number of vertices to use on the second gate.
num_points = 4

# Opens the interactive gate drawing tool.
# Use test_samples to plot all 'A' samples together, or test_sample to plot just the first sample (may be faster).
vertex_control(test_samples, scatter_axes_2, num_points, GATE_RESULTS, 'scatter_gate_2', log=False)

### 1(b). Defining the Gates

In [ ]:
# Creates the main scatter gate using the interactive results. Set the gate_vertices variables to None 
# if no polygon gate is desired.
gate_vertices_1 = GATE_RESULTS.get('scatter_gate_1', None)
gate_vertices_2 = GATE_RESULTS.get('scatter_gate_2', None)
scatter_gates = []

if gate_vertices_1 is not None:
    scatter_gates.append(PolyGate(gate_vertices_1, scatter_axes_1, region='in', name='scatter_gate_1'))
    print("Gate 1 created with vertices: {}".format([tuple(row) for row in gate_vertices_1]))
if gate_vertices_2 is not None:
    scatter_gates.append(PolyGate(gate_vertices_2, scatter_axes_2, region='in', name='scatter_gate_2'))
    print("Gate 2 created with vertices: {}".format([tuple(row) for row in gate_vertices_2]))
    
if gate_vertices_1 is None and gate_vertices_2 is None:
    print("No polygon gates will be used.")

# Optionally, create some threshold gates to further filter the data.
scatter_threshold_gates = [
    # ThresholdGate(0, 'FSC-H', region='above'),
    # ThresholdGate(9000, 'SSC-H', region='below'),
    # ThresholdGate(300, 'SSC-H', region='above')
]

def gate_by_scatter(sample):
    """Gates the given FCMeasurement using the scatter_gate and the list of scatter_threshold_gates. 
    Returns the new gated sample."""
    tsample = sample
    for gate in scatter_gates + scatter_threshold_gates:
        tsample = tsample.gate(gate)
    return tsample

In [ ]:
# Show several examples from the first titration with this gate
files = get_titration_files(TITRATIONS[0])
gates_to_show = [x for x in scatter_gates + scatter_threshold_gates if all([c in scatter_axes for c in x.channels])]
plt.figure(figsize=(16, 10))

for i, path in enumerate(files[:24]):
    plt.subplot(4, 6, i + 1)
    
    # Get the sample, gate it, and plot it
    sample = get_sample(path, "Sample {}".format(i), transform=True)
    filtered_sample = gate_by_scatter(sample)
    filtered_sample.plot(scatter_axes, ax=plt.gca(), kind='scatter', color='r', s=1, gates=gates_to_show)
    
    plt.title(os.path.splitext(os.path.basename(path))[0])
    
plt.tight_layout()
plt.show()

## Step 2. Fluorescence Gating

In [ ]:
# Gate the test samples from above
fluor_samples = [gate_by_scatter(samp) for samp in test_samples]
fluor_sample = fluor_samples[0]

In [ ]:
# Take a look at the first sample, and all the samples
plt.figure()
fluor_sample.plot(fluor_axes, ax=plt.gca(), kind='scatter', color='r', s=1);
plt.figure()
for sample in fluor_samples:
    sample.plot(fluor_axes, ax=plt.gca(), kind='scatter', color='r', s=1);

### 2(a). Drawing Gates

The following cell opens the interactive gate drawing tool for the binding and expression axes. If no polygon gate is desired, simply skip to section 2(b). Alternatively, if you already know the vertices for the polygon you want, skip to section 2(b) and replace the value of `gate_vertices` with the list of coordinates.

In [ ]:
# Choose the number of vertices to use on the gate.
num_points = 3

# Use fluor_samples to plot all 'A' samples together, or fluor_sample to plot just the first sample (may be faster).
vertex_control(fluor_samples, fluor_axes, num_points, GATE_RESULTS, 'fluor_gate', log=False)

### 2(b). Defining the Gates

In [ ]:
# Creates the main polygon fluorescence gate using the interactive results. 
# Set gate_vertices to None if no polygon gate is desired.
gate_vertices = GATE_RESULTS.get('fluor_gate', None)

if gate_vertices is not None:
    fluor_gate = PolyGate(gate_vertices, fluor_axes, region='in', name='fluor_gate')
    print("Gate created with vertices: {}".format([tuple(row) for row in gate_vertices]))
else:
    fluor_gate = None
    print("No polygon gate will be used.")


# Create some threshold gates to further filter the data.
fluor_threshold_gates = [
    # ThresholdGate(1, 'PE-A', region = 'above'),
    # ThresholdGate(4000, 'Alexa Fluor 680-A', region = 'above'),
]

def gate_by_fluorescence(sample):
    """Gates the given FCMeasurement using the fluor_gate and the list of fluor_threshold_gates. 
    Returns the new gated sample."""
    tsample = sample
    if fluor_gate is not None:
        tsample = sample.gate(fluor_gate)
    for gate in fluor_threshold_gates:
        tsample = tsample.gate(gate)
    return tsample

In [ ]:
# Show several examples from the first titration with all gates applied
files = get_titration_files(TITRATIONS[0])
gates_to_show = ([fluor_gate] if fluor_gate is not None else []) + fluor_threshold_gates
plt.figure(figsize=(16, 10))

for i, path in enumerate(files[:24]):
    plt.subplot(4, 6, i + 1)
    
    # Get the sample, gate it, and plot it
    sample = get_sample(path, "Sample {}".format(i), transform=True)
    filtered_sample = gate_by_fluorescence(gate_by_scatter(sample))
    filtered_sample.plot(fluor_axes, ax=plt.gca(), kind='scatter', color='r', s=1, gates=gates_to_show)
    
    plt.title(os.path.splitext(os.path.basename(path))[0])
    
plt.tight_layout()
plt.show()

## Step 3. $K_d$ Estimation

Up until now we have only been working with a small subset of the data - now, we will perform the analysis on all samples. 

**A note on transformations:** All of the gates we have created are in hyperlog-space, so this script is careful to apply any gates in that space. The best practice for fitting, however, is usually to perform the fit on linear (un-transformed) medians. If the `linear_medians` variable below is set to `True`, the median of the un-transformed data will be used (still gated in log space); otherwise, the median of the transformed data will be used.

In [101]:
# If this is set to True, we will use the untransformed data to get the medians; 
# otherwise, we use the log-transformed data
linear_medians = True

In [ ]:
# Get the medians of every gated sample
medians = np.zeros((len(TITRATIONS), len(CONCENTRATIONS)))

for i, titration in enumerate(TITRATIONS):
    for j, sample_info in enumerate(titration):
        # Get the path for this sample
        path = get_sample_path(*sample_info)
        id_name = os.path.splitext(os.path.basename(path))[0]

        # Load the sample and gate it
        sample = get_sample(path, id_name, transform=False)
        tsample = transform_sample(sample)
        filtered_tsample = gate_by_fluorescence(gate_by_scatter(tsample))
        filtered_sample_data = sample.get_data().loc[filtered_tsample.get_data().index]

        # Compute median of expression axis values
        if linear_medians:
            medians[i, j] = filtered_sample_data[expression_axis].median()
        else:
            medians[i, j] = filtered_tsample.data[expression_axis].median()

In [ ]:
# Plot the medians - pick a random titration to look at
titration_to_plot = 0
plt.figure(figsize=(16, 6))

for sample_num, sample_info in enumerate(TITRATIONS[titration_to_plot]):
    # Get the path for this sample
    path = get_sample_path(*sample_info)
    id_name = os.path.splitext(os.path.basename(path))[0]
    median = medians[titration_to_plot, sample_num]

    # Load the sample and gate it
    sample = get_sample(path, id_name, transform=True)
    filtered_sample = gate_by_fluorescence(gate_by_scatter(sample))
    
    # Plot
    plt.subplot(2, int(ceil(len(CONCENTRATIONS) / 2.0)), sample_num + 1)
    filtered_sample.plot(fluor_axes, ax=plt.gca(), kind='scatter', color='r', s=1)
    # The median should be hlog-transformed for display if it's on a linear scale
    plt.hlines(hlog(median, b=HYPERLOG_B) if linear_medians else median, *plt.xlim(), color='b')
    plt.title(id_name)
    
plt.tight_layout()
plt.show()

In [ ]:
# Now we compute all the Kds. 
# Initial parameters:
init_lower = 0
init_upper = 4000
init_kd = 1
plot_fits = True

# List of dictionaries of fit results
fit_results = []

if plot_fits: plt.figure(figsize=(16, 16))
for i, (label, titration, dataset) in enumerate(zip(LABELS, TITRATIONS, medians)):

    if plot_fits:
        plt.subplot(int(ceil(len(TITRATIONS) / 4.0)), 4, i + 1)

    # Compute fit and save to results dictionary
    kd, sat, init, err, chisqr, r2 = run_lmfit(CONCENTRATIONS, dataset, init_lower, init_upper, init_kd, plot_fits)
    result = [("label", label), 
              ("kd", kd), ("sat", sat), ("init", init),
              ("err", err), ("chisqr", chisqr), ("r2", r2)]
    for conc, val in zip(CONCENTRATIONS, dataset):
        result.append(("conc_{}".format(conc), val))
    fit_results.append(OrderedDict(result))

    if plot_fits:
        plt.title("{} (Kd = {:.3g})".format(label, kd))
            
if plot_fits:
    plt.tight_layout()
    plt.show()

In [ ]:
# Build a dataframe from the results list
results_df = pd.DataFrame(fit_results)
results_df

In [ ]:
# Choose an output path, and write out to CSV
out_path = "kd_list.csv"
results_df.to_csv(out_path, index=False)

That's all, folks!